In [2]:
import torch
from data.data import *
from train import load_checkpoint
from cfg.configs import Config

nncfg = Config("nn_vls_dense.yaml")
model_dir = nncfg.config['model']['log_dir']
ds = nncfg.get_dataset()
%load_ext autoreload
%autoreload 2

transforming inputs
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


transforming inputs


In [3]:
nncfg = Config("nn_vls_attn.yaml")
model = nncfg.get_model()
model(ds.phys_prep[:10]).shape

TypeError: unsupported operand type(s) for +: 'Tensor' and 'tuple'

In [4]:
attn = torch.nn.MultiheadAttention(128, 8)

In [12]:
res = attn(torch.ones(1, 3,128), torch.ones(1, 3,128), torch.ones(1, 3,128), need_weights=False)

In [13]:
res[1]

In [14]:
ds.phys_prep[:10].shape

torch.Size([10, 3])

torch.Size([10, 3])

In [7]:
a.max()

tensor(1.0000)

In [14]:
ckpt = load_checkpoint(f"{model_dir}/ckpt_2499_-001_2.15e-02", nncfg)
model = ckpt['model']
example = torch.rand(2, 3)
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save(f"{model_dir}/traced_2499_1.pt")

In [3]:
ckpt = load_checkpoint(f"{model_dir}/ckpt_7298_-001_8.76e-06", nncfg)
model = ckpt['model']
example = torch.rand(2, 3)
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save(f"{model_dir}/traced_7300_1.pt")

In [4]:
ds = nncfg.get_dataset()

transforming inputs


In [6]:
stand = ds.inpp.get_params()['steps'][0][1]
norm = ds.inpp.get_params()['steps'][1][1]

In [13]:
norm.get_params()

{'copy': True, 'feature_range': (0, 1)}

In [1]:
params = [stand.mean_, stand.scale_, norm.data_max_, norm.data_min_]

NameError: name 'stand' is not defined